# HW 3: Spam vs. Ham

In this homework assignment, you will implement, train, and test a naïve Bayes spam classifier on the [Enron Spam Dataset](https://www.kaggle.com/datasets/marcelwiechmann/enron-spam-data/data) [(Metsis et al., 2006)](https://www2.aueb.gr/users/ion/docs/ceas2006_paper.pdf).

## Problem 0: Setup (0 Points in Total)

Before beginning the assignment, you will need to download and examine the [Enron Spam Dataset](https://www.kaggle.com/datasets/marcelwiechmann/enron-spam-data/data), a dataset for spam classification created by [Metsis et al. (2006)](https://www2.aueb.gr/users/ion/docs/ceas2006_paper.pdf).

### Problem 0a: Download Spam Dataset (No Submission, 0 Points)

Please download the Enron Spam Dataset from the Assignments folder of Blackboard. The dataset consists of a single file, `enron_spam_data.csv`. Please put this file in the `data` subdirectory of this folder.

### Problem 0b: Inspect Spam Dataset (No Submission, 0 Points)

Please look at the `enron_spam_data.csv` file and examine its structure. You can open the file using Jupyter Notebook, Microsoft Excel, Google Sheets, or any plaintext editor (e.g., Notepad in Windows, TextEdit in Mac OS, or Text Editor in Linux). 

Notice that each line of this file represents an email. What information does the file contain about each email?

Notice that the subject and message of each email have already been tokenized, with the tokens separated by spaces.

## Problem 1: Python Exercises (5 Points in Total)

In these exercises, you will learn and practice Python concepts needed in order to implement and test a naïve Bayes spam classifier in Problems 2 and 3.

### Problem 1a: Understand Dict Disjunction (Written, 1 Point)

Try running the following code.

In [1]:
d1 = {"a": 1, "b": 2}
d2 = {"c": 3, "d": 4}
d1 | d2

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

In general, if `d1` and `d2` are `dict`s, what is `d1 | d2`?

### Problem 1b: Understand CSV Files (Written, 1 Point)

Look at the file `data/sample_data.csv`. Please write down the data contained in this file, in the form of a table. Your table should look like the following:

| col1 | col2 | col3 |
|------|------|------|
|   |   |   |
|   |   |   |
|   |   |   |
|   |   |   |

### Problem 1c: Understand the `csv` Module (No Submission, 0 Points)

Try running the following code, and pay attention to what it does.

In [2]:
import csv

with open("data/sample_data.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['col1', 'col2', 'col3']
['1', '2', '3']
['4', '5', '6']
['7', '8', '9']
['10', '11', '12']


### Problem 1d: Skip Header Row (Written, 1 Point)

The following cell is similar to the one from Problem 1c, but contains an extra line of code. What is this extra line of code, and what does it do?

In [3]:
with open("data/sample_data.csv", "r") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        print(row)

['1', '2', '3']
['4', '5', '6']
['7', '8', '9']
['10', '11', '12']


**Hint 1:** Look at the title of this problem.

**Hint 2:** Let's try looking at the contents of the `header` variable:

In [4]:
header

['col1', 'col2', 'col3']

### Problem 1e: Load Data from CSV File (Code, 1 Point)

Please implement the function `load_csv` in `hw3.py`. This function should load the data from `data/sample_data.csv` in the form of a `dict`:

In [5]:
import hw3

hw3.load_csv("data/sample_data.csv")

{'col1': ['1', '4', '7', '10'],
 'col2': ['2', '5', '8', '11'],
 'col3': ['3', '6', '9', '12']}

**Hint:** You may find it helpful to look at Problem 1f, though you don't have to.

### Problem 1f: Remember the `zip` Function (No Submission, 0 Points)

Recall that the `zip` function combines two `list`s into a `list` of `tuple`s:

In [6]:
l1 = ["a", "b", "c"]
l2 = [1, 2, 3]
list(zip(l1, l2))

[('a', 1), ('b', 2), ('c', 3)]

Recall that you can "unzip" a list of tuples by using `zip` with a `*`:

In [7]:
l3 = [("a", 1), ("b", 2), ("c", 3)]
list(zip(*l3))

[('a', 'b', 'c'), (1, 2, 3)]

### Problem 1g: Understand Variable Unpacking (Written, 1 Point)

In each of the following code cells, what do the variables `x`, `y`, and `z` contain?

In [8]:
# Code cell 1
x, y, z = 1, 2, 3

In [9]:
# Code cell 2
x, y, z = l3

In [10]:
# Code cell 3
for x, y in zip(l1, l2):
    print(x, y, sep=", ")

a, 1
b, 2
c, 3


In general, how do you set multiple variables at the same time, in one line of code?

## Problem 2: Implement Spam Classifier (12 Points in Total)

In this problem, you will complete the implementation of a naïve Bayes spam classifier.

### Problem 2a: Deserialize Spam Dataset (Code, 2 Points)

Please implement the function `hw3.load_spam_data`, which loads the Enron Spam Dataset into Python. The process of converting data from a file into Python objects is called [_deserialization_](https://en.wikipedia.org/wiki/Serialization).

In [11]:
# Deserialize the Enron Spam Dataset
emails = hw3.load_spam_data("data/enron_spam_data.csv")

The function should represent the dataset as a `list` of emails, where each email is represented as a `dict` with the following information:
```python
{"subject": ["Office", "Hours", "?"],  # The subject of the email
 "message": ["Hi", "Yulu", ",", "I", "need", "help", ...],  # The contents of the email
 "label": "ham"}
```

For example:

In [12]:
# Look at an email
emails[2]

{'subject': ['calpine', 'daily', 'gas', 'nomination'],
 'message': ['-', 'calpine', 'daily', 'gas', 'nomination', '1', '.', 'doc'],
 'label': 'ham'}

**Hint 1:** Use the `String.split` function:

In [13]:
"Hello world !".split()

['Hello', 'world', '!']

**Hint 2:** In order to load the emails, you will need to run the following code:

In [14]:
import sys

csv.field_size_limit(sys.maxsize)

9223372036854775807

The `csv` module has a limit on the length of each column of a CSV file. Some of the emails in the Enron Spam Dataset are longer than the default maximum length, so the above code sets the length limit to the maximum possible value. The `hw3.py` file already has this code inside of it, but you will need to run the above cell if you choose to write a version of your function in this notebook.

### Problem 2b: Extract Common Token Types (Code, 2 Points)

Please implement the function `hw3.get_vocab`, which extracts the $n$ most common token types (i.e., words) in a list of emails.

In [15]:
# The 10 most common token types in email subjects
hw3.get_vocab(emails, "subject", n=10)

{'.', '/', ':', ';', 'date', 'fw', 'hour', 'hourahead', 're', 'start'}

In [16]:
# The 10 most common token types in email messages
hw3.get_vocab(emails, "message", n=10)

{',', '-', '.', '/', ':', 'a', 'and', 'of', 'the', 'to'}

If no argument is provided for the parameter `n`, then all token types should be extracted.

In [17]:
# How many unique tokens appear in email subjects?
len(hw3.get_vocab(emails, "subject"))

7543

### Problem 2c: Extract Features from Emails (Code, 2 Points)

Please implement the function `hw3.get_features`, which extracts binary unigram features from an email.

There are two kinds of features that might be extracted:
- `subject_contains(w)`, which indicates whether the email's subject contains at least one `w`
- `message_contains(w)`, which indicates whether the email's message contains at least one `w`.

The features of an email are represented as a `dict` in the following format:
```python
{"subject_contains(Re:)": True,
 "subject_contains(Fwd:)": False,
 "subject_contains(Hello)": True,
 ...,
 "message_contains(The)": True,
 "message_contains(of)": True,
 "message_contains(in)": False,
 ...}
```
where the keys of the `dict` (i.e., the possible features of the email) are indicated by the parameters `subj_vocab` and `msg_vocab`.

For example, the following code extracts binary unigram features representing the five most common token types in email subjects and messages.

In [18]:
# Use the 5 most common subject and message tokens
subj_vocab = hw3.get_vocab(emails, "subject", n=5)
msg_vocab = hw3.get_vocab(emails, "message", n=5)

In [19]:
# Extract binary unigram features from an email
hw3.get_features(emails[1], subj_vocab, msg_vocab)

{'subject_contains(.)': True,
 'subject_contains(/)': False,
 'subject_contains(;)': False,
 'subject_contains(re)': False,
 'subject_contains(:)': False,
 'message_contains(.)': True,
 'message_contains(:)': True,
 'message_contains(,)': True,
 'message_contains(the)': True,
 'message_contains(-)': True}

### Problem 2d: Preprocess Datasets (Code, 2 Points)

The `hw3.SpamClassifier` class represents a naïve Bayes spam classifier. We construct a `SpamClassifier` by specifying what subject and message features we would like to use.

In [20]:
# Construct a SpamClassifier using 5 subject and message features
classifier = hw3.SpamClassifier(subj_vocab, msg_vocab)

Please implement the method `hw3.SpamClassifier.preprocess`, which converts a `list` of emails into the features of those emails, along with the classification label (`"ham"` or `"spam"`) of each email. Usage examples are shown below.

In [21]:
# Preprocess some ham emails
classifier.preprocess(emails[:3])

[({'subject_contains(.)': False,
   'subject_contains(/)': False,
   'subject_contains(;)': False,
   'subject_contains(re)': False,
   'subject_contains(:)': False,
   'message_contains(.)': False,
   'message_contains(:)': False,
   'message_contains(,)': False,
   'message_contains(the)': False,
   'message_contains(-)': False},
  'ham'),
 ({'subject_contains(.)': True,
   'subject_contains(/)': False,
   'subject_contains(;)': False,
   'subject_contains(re)': False,
   'subject_contains(:)': False,
   'message_contains(.)': True,
   'message_contains(:)': True,
   'message_contains(,)': True,
   'message_contains(the)': True,
   'message_contains(-)': True},
  'ham'),
 ({'subject_contains(.)': False,
   'subject_contains(/)': False,
   'subject_contains(;)': False,
   'subject_contains(re)': False,
   'subject_contains(:)': False,
   'message_contains(.)': True,
   'message_contains(:)': False,
   'message_contains(,)': False,
   'message_contains(the)': False,
   'message_contain

In [22]:
# Preprocess some spam emails
classifier.preprocess(emails[-2:])

[({'subject_contains(.)': True,
   'subject_contains(/)': False,
   'subject_contains(;)': False,
   'subject_contains(re)': True,
   'subject_contains(:)': True,
   'message_contains(.)': True,
   'message_contains(:)': True,
   'message_contains(,)': True,
   'message_contains(the)': True,
   'message_contains(-)': True},
  'spam'),
 ({'subject_contains(.)': True,
   'subject_contains(/)': False,
   'subject_contains(;)': False,
   'subject_contains(re)': True,
   'subject_contains(:)': True,
   'message_contains(.)': True,
   'message_contains(:)': True,
   'message_contains(,)': True,
   'message_contains(the)': True,
   'message_contains(-)': True},
  'spam')]

### Problem 2e: Evaluate Spam Classifier (Code, 4 Points)

Please implement the method `hw3.SpamClassifier.evaluate`, which computes the `SpamClassifier`'s accuracy, precision, recall, and $F_1$ score on a list of emails. The results of the evaluation should be reported as a `dict` in the following format:
```python
{"accuracy": .78,  # These numbers are fake
 "precision": .727,
 "recall": .851,
 "f1": .784}
```

Recall that the definitions of precision, recall, and $F_1$ score require one of the classes to be defined as the "positive label" and the other classes to be defined as the "negative label." For the `SpamClassifier.evaluate` method, the positive label is indicated by the class-level property `SpamClassifier.positive`.

In [23]:
hw3.SpamClassifier.positive

'spam'

The `SpamClassifier` has a property called `.model`, which the constructor (i.e., the `SpamClassifier.__init__` method) initializes to `None`. When the method `SpamClassifier.train` is called, an `nltk.NaiveBayesClassifier` is trained on the dataset provided, and the trained model is saved to `SpamClassifier.model`. This is the model your `SpamClassifier.evaluate` method should evaluate.

To illustrate how the method works, let's now look at a usage example.

In [24]:
# Construct a small train and test set
import random

random.seed(10)
random.shuffle(emails)

train = emails[:100]
test = emails[100:200]

In [25]:
# Train a naive Bayes classifier
classifier.train(train)

# Notice that the naive Bayes classifier is saved to classifier.model
classifier.model

In [26]:
# Evaluate the trained naive Bayes classifier
classifier.evaluate(test)

{'accuracy': 0.73,
 'precision': 0.7636363636363637,
 'recall': 0.75,
 'f1': 0.7567567567567568}

In [27]:
# Let's confirm that the accuracy matches NLTK's built-in code for accuracy
import nltk

nltk.classify.accuracy(classifier.model, classifier.preprocess(test))

0.73

**Hint 1:** Let $\textsf{tp}$, $\textsf{tn}$, $\textsf{fp}$, and $\textsf{fn}$ be the number of true positive, true negative, false positive, and false negative predictions made by a model on a test set, respectively. The accuracy, precision, recall, and F1 scores can be calculated as follows.
$$\begin{align*}
    \text{accuracy} &= \frac{\textsf{tp} + \textsf{tn}}{\textsf{tp} + \textsf{tn} + \textsf{fp} + \textsf{fn}} \\
    \text{precision} &= \frac{\textsf{tp}}{\textsf{tp} + \textsf{fp}} \\
    \text{recall} &= \frac{\textsf{tp}}{\textsf{tp} + \textsf{fn}} \\
    F_1 &= \frac{2\textsf{tp}}{2\textsf{tp} + \textsf{fp} + \textsf{fn}}
\end{align*}$$

**Hint 2:** Recall that you can add a `bool` to an `int`:

In [28]:
3 + True  # True == 1

4

In [29]:
2 + False  # False == 0

2

**Hint 3:** You can use the `nltk.NaiveBayesClassifier.classify` method to predict a label using the model, like so:

In [30]:
# Take an email from the test set and preprocess it
test_email = classifier.preprocess(test[0:1])
x, y = test_email[0]

# Predict a label for x
classifier.model.classify(x)

'spam'

In [31]:
# Was it correct?
y

'ham'

**Hint 4:** Look at Problems 1f and 1g.

**Hint 5:** If you'd like, you may use any built-in NLTK functions you find that compute the four metrics. However, it's probably easier just to compute the metrics yourself. Python libraries other than NLTK and the default libraries may or may not be available on the Gradescope autograder.

## Problem 3: Test Spam Classifier (3 Points in Total)

Once you have finished implementing the `SpamClassifier`, you will test your classifier using the full Enron Spam Dataset.

### Problem 3a: Run Experiment (No Submission, 1 Point)

Please train and evaluate your `SpamClassifier` by running `hw3.py` as a script. To do so, navigate to this directory using Terminal (Mac OS/Linux) or Anaconda Prompt (Windows), and run the following code:
```bash
python hw3.py
```

Alternatively, you can run `hw3.py` from this notebook by running the following code cell:

In [32]:
!python hw3.py

The `hw3.py` script splits the Enron Spam Dataset into training, development, and test sets. It will train three different `SpamClassifiers`:
- a model that uses 500 subject features and 2000 message features
- a model that uses the subject features only
- a model that uses the message features only.

The three models are then evaluated on the development set, and the model that achieves the highest $F_1$ score is evaluated on the test set.

**Hint 1:** The script should take roughly 15 minutes to run.

**Hint 2:** You do not need to submit anything for this problem, and you do not need to declare or provide evidence of actually having run the script. You will receive full credit for this problem no matter what.

### Problem 3b: Report Results (Written, 1 Point)

Please report the results of the experiment by filling in the following table.

| Model Type | Dataset | Accuracy | Precision | Recall | F1 |
|------------|---------|----------|-----------|--------|----|
| Subject + Message | Dev |   |   |   |   |
| Subject Only      | Dev |   |   |   |   |
| Message Only      | Dev |   |   |   |   |
|                   | Test |   |   |   |   |

### Problem 3c: Analyze Results (Written, 1 Point)

Please answer the following questions, based on the results reported in Problem 3b as well as the 5 most informative features of each model.
- Which part of the email is more useful for spam classification: the subject or the message?
- If a spam classifier already has access to the message of an email, is there much additional benefit to also having the email's subject?
- It seems that there are many features that provide evidence that an email is spam, but few or no features that provide evidence that an email is ham. Why do you think this is?